In [1]:
" 
Details about this code   ;   - exporting all genes and pathways information from hipathia 
                                and stores genes (txt format) and pathways (graphml format) in specified location
                              - exporting the conversion table which is between entrez and gene symbol
Exported folder location ;  
  1.  ../data/01_R_o_pathways > stores all pathways (graphml format)
  2.  ../data/01_R_o_genes    > stores genes list (txt format) and conversion table (between entrez and gene saymbol)
"

[1] " \nDetails about this code   ;   - exporting all genes and pathways information from hipathia \n                                and stores genes (txt format) and pathways (graphml format) in specified location\n                              - exporting the conversion table which is between entrez and gene symbol\nExported folder location ;  \n  1.  ../data/01_R_o_pathways > stores all pathways (graphml format)\n  2.  ../data/01_R_o_genes    > stores genes list (txt format) and conversion table (between entrez and gene saymbol)\n"

In [1]:
library('hipathia') # hipathia package
# library('igraph') # converting graph to graphml format
# library('org.Mm.eg.db') #mmu, converting entrez to gene symbol
# library('AnnotationDbi')

Loading required package: igraph

Attaching package: ‘igraph’

The following objects are masked from ‘package:stats’:

    decompose, spectrum

The following object is masked from ‘package:base’:

    union

Loading required package: AnnotationHub
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:igraph’:

    normalize, path, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lappl

In [2]:
folder_output_pathways = paste0('../data/hipathia_pathways/')
folder_output_pathway_genes = paste0('../data/hipathia_genes_detail/')
folder_output_list = paste0('../data/pathways/')

# dir.create(folder_output_pathways, showWarnings = FALSE)
# dir.create(folder_output_pathway_genes, showWarnings = FALSE)

pathways <- load_pathways(species = 'mmu')
# pathways_list <- get_pathways_list(pathways)
# get_path_names(pathways)

Using temporary cache /tmp/RtmpPnbKWN/BiocFileCache
snapshotDate(): 2019-10-29
Using temporary cache /tmp/RtmpPnbKWN/BiocFileCache
Using temporary cache /tmp/RtmpPnbKWN/BiocFileCache
Loaded 142 pathways


In [4]:
df_h_final <- read.table('../data/pathways/hipathia_pathways_final.csv', sep=',', header=TRUE)

df_pathways = data.frame(pathways$all.labelids)
print(paste0('Pathway number    , ', nrow(df_h_final)))
print(paste0('All sub-pathways  , ', nrow(df_pathways)))
df_pathways = merge(df_h_final, df_pathways)
print(paste0('Final sub-pathways, ', nrow(df_pathways)))
list_path_id = levels(df_pathways$path.id)
# print(list_path_id[1])
# list_path_id

[1] "Pathway number    , 92"
[1] "All sub-pathways  , 6789"
[1] "Final sub-pathways, 4528"


In [19]:
# EXPORTING GENE LISTS

# Creating dataframe which store all genes + info rows
df_gene <- data.frame(pathways$all.genes)
colnames(df_gene) = c('entrez')
# head(df_gene)
l_gene_final<-c()
#GENES
for (all_pathways_ in c(1:length(list_path_id)) ){
    df_merge = df_gene
    l_main<-c()
#     print(list_path_id[all_pathways_])
    length_subpathways = length(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs)
    for (sub_pathways_ in c(1:length_subpathways)) {
        l_sub<-c()
#         print(sub_pathways_)
        genes_circuits = V(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[[sub_pathways_]])$genesList
        sub_path_name = names(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[sub_pathways_][1])
#         print(sub_path_name)
        for (sub_circuits in c(1:length(genes_circuits))){
            for (genes_ in c(1:length(genes_circuits[sub_circuits][1][[1]]))){
                gene_value = genes_circuits[sub_circuits][1][[1]][genes_]
                if (!is.na(gene_value) && gene_value != '/' && (gene_value == 'NA') == FALSE){
                    l_sub <-append(l_sub,gene_value)
                }
            }
            l_sub = unique(l_sub)
        }
        if (length(l_sub) != 0) {
#             Combining all genes which are using in specified pathway
            l_main<-append(l_main, l_sub)
            df_temp <- data.frame(l_sub, 1)
            names(df_temp) = c('entrez', sub_path_name)
        }
#         Inner join
        df_merge = merge(df_temp,df_merge, by='entrez', all=T)
    }
    l_main = unique(l_main)
#     Assigning all NA's as 0
#     df_merge[s.na(df_merge)] <- 0
    df_merge[!is.na(df_merge)] 
#     x[!is.na(x)]
    indices_genes <- as.vector(which(df_merge$entrez %in% l_main, arr.ind = TRUE))
#     1L * df_merge$entrez %in% c('info', l_main)
    df_path_genes <- (df_merge[c(indices_genes), ])
    rownames(df_path_genes) <- 1:nrow(df_path_genes)
    l_gene_final = append(l_gene_final, l_main)
#     print(length(l_gene_final))
#     Exporting txt file belgons to each pathways (142 txt file for mmu)
    write.table(df_path_genes, paste0(folder_output_pathway_genes,list_path_id[all_pathways_],'_gene_list.txt'), sep=',', row.names=FALSE)
}
    
write.table(df_gene$entrez[(df_gene$entrez) != 'NA'], paste0(folder_output_pathway_genes,'hipathia_all_gene_list.csv'),sep=',',row.names = FALSE)
write.table(unique(l_gene_final), paste0(folder_output_list,'hipathia_gene_list.csv'),sep=',', row.names = FALSE)
print(paste0('THE GENES LIST EXPORTED!!! - ', folder_output_pathway_genes,'hipathia_all_gene_list.csv and', folder_output_list,'hipathia_gene_list.csv'))

[1] "THE GENES LIST EXPORTED!!! - ../data/hipathia_genes_detail/hipathia_all_gene_list.csv and../data/pathways/hipathia_gene_list.csv"


In [20]:
# EXPORTING PATHWAYS

for (all_pathways_ in c(1:length(list_path_id)) ){
#     print(list_path_id[all_pathways_])
    dir.create(paste0(folder_output_pathways,list_path_id[all_pathways_]), showWarnings = FALSE )
    write_graph(pathways$pathigraphs[[list_path_id[all_pathways_]]]$graph, paste0(folder_output_pathways,list_path_id[all_pathways_],"/", list_path_id[all_pathways_],"_00_main.graphml"), format = c("graphml"))
    length_subpathways = length(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs)
    for (sub_pathways_ in c(1:length_subpathways)) {
        sub_path_name = names(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[sub_pathways_][1])
#         print(path_name)
        write_graph(pathways$pathigraphs[[list_path_id[all_pathways_]]]$effector.subgraphs[[sub_pathways_]], paste0(folder_output_pathways,list_path_id[all_pathways_],"/", sub_path_name,".graphml"), format = c("graphml"))
    }
}
print("PATHWAYS EXPORTED!!")

[1] "PATHWAYS EXPORTED!!"
